## Summary Data for GraphML Files

In [1]:
import graph_tool.all as gt
import os
import numpy as np
import graph_tool
from tqdm import tqdm_notebook

from lib.analysis import clustering
from lib.analysis import dist
from lib.analysis import model
from lib.analysis import power
from lib.analysis import stats

## Part 1: Summary Statistics

Basic statistics for each country's government

In [2]:
data = 'data'
ignore = ['.DS_Store', 'us']
graphs = {}

In [3]:
for country in [d for d in os.listdir(data) if d not in ignore]:
    path = os.path.join(data, country)
    for file in [f for f in os.listdir(path) if f not in ignore]:
        if file.endswith('.graphml'):
            fpath = os.path.join(path, file)
            graphs[file[:-8]] = gt.load_graph(fpath)

In [4]:
with open("summary.csv", "w+") as f:
    f.write(','.join(['Number of Edges', 'Number of Vertices',
                      'Clustering Coefficient', 'Average Degree',
                      'Average Excess Degree', 'Average Distance',
                      'Size of LCC', 'Number of CC']) + '\n')
    for name, g in graphs.items():
        print(name)
        f.write(f'{name},{str(stats.summary(g))}\n')

net_se
net_fr_se
net_fr_an
net_ch_cs
net_ch_cn
uk
net_dk
net_at


## Part 2: Assortivity

Computes assoritivty on specified attribute for every country across years 

In [2]:
individual_graphs = {}
root = 'data/'
countries = os.listdir(root)
nets = [country + "/" + net + "/" for country in countries
                            for net in os.listdir(root + country)
                            if not net.endswith('.graphml')]
for net in nets:
    for x in os.listdir(root+net):
        if x.endswith('.graphml'):
            individual_graphs[x] = gt.load_graph(root + net + x) 

In [3]:
attribute = 'party'
label = 'party_id'
with open("assortivity_summary.csv", "w+") as f:
    f.write(','.join(['Assortivity Measure']) + '\n')
    for name, g in individual_graphs.items():
        if attribute in g.vertex_properties:
            print(name, attribute)
            g.set_directed(False)
            gt.remove_parallel_edges(g)
            stats.generate_party_property(g, attribute)
            f.write(f'{name},{str(stats.assortativity_summary(g, label))}\n')


net_se2014-2018.graphml party
net_se2010-2014.graphml party
net_se2006-2010.graphml party
net_se1998-2002.graphml party
net_se2002-2006.graphml party
net_se1988-1991.graphml party
net_se1994-1998.graphml party
net_se1991-1994.graphml party
net_fr_an2007-2012.graphml party
net_fr_an2002-2007.graphml party
net_fr_an1986-1988.graphml party
net_fr_an2012-2017.graphml party
net_fr_an1997-2002.graphml party
net_fr_se1997-2002.graphml party
net_fr_se2012-2017.graphml party
net_fr_se1986-1988.graphml party
net_fr_se1988-1993.graphml party
net_fr_se1993-1997.graphml party
net_fr_se2002-2007.graphml party
net_fr_se2007-2012.graphml party
net_ch_cs1995-1999.graphml party
net_ch_cs2003-2007.graphml party
net_ch_cs2011-2015.graphml party
net_ch_cs1999-2003.graphml party
net_ch_cs2007-2011.graphml party
net_ch_cn2003-2007.graphml party
net_ch_cn1999-2003.graphml party
net_ch_cn2007-2011.graphml party
net_ch_cn2011-2015.graphml party
net_ch_cn1995-1999.graphml party
net_dkEconomy.graphml party
net_dk